# CS182 HW3 Coding [40 points]

In this coding homework, you will be required to complete several models for binary classification and try to find the inplicit relationship of them by yourself. 

**Good luck!**


In [1]:
from sklearn import svm
from scipy import special
import numpy as np

In [2]:
X_train = np.loadtxt('data/X_train.txt')
X_val = np.loadtxt('data/X_val.txt')
X_test = np.loadtxt('data/X_test.txt')
y_train = np.loadtxt('data/y_train.txt')
y_val = np.loadtxt('data/y_val.txt')
y_test = np.loadtxt('data/y_test.txt')

w = np.loadtxt('data/w.txt')
w0 = np.loadtxt('data/w0.txt')

## (a) Simple Perceptron

(1) Activation functions and loss functions are important parts of each neural network, and there are multiple ways of calculating them. 

 **[3 points]** In this question, we ask you to implement the **sigmoid function** and **binary cross entroy loss function** serving for the binary classification.

In [3]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

def BCEloss(y_pred, y):
    y_new = y*np.log(y_pred)+(1-y)*np.log(1-y_pred)
    sum = np.sum(y_new)
    return -sum

(2) **[3 points]** In this question, we ask you to implement the **softmax function** and **cross entroy loss function** serving for the multiple classification.

In [4]:
def softmax(x):
    exp_x = np.exp(x)
    sum_exp_x = np.sum(exp_x)
    return exp_x/sum_exp_x
def cross_entropy_loss(y,y_pre):
    y_new = y*np.log(y_pre)
    sum = np.sum(y_new)
    return -sum


(3) **[10 points]** Learning a simple perceptron with **batch GD** (using the given initializations $w^{init}$ and $w^{init}_{0}$) based on the training set ($X_{train}$, $y_{train}$): use the training set and the validation set to obtain a good learning rate (you can set the maximum for iterations to 50 and try different learning rate in [$10^{−4}$,  $10^{−8}$] ); output the learned model and evaluate its performance on the test set with the classification accuracy.

In [41]:
# BGD Implementation
def cal_gradient(theta, X, Y):
    diff = sigmoid(np.dot(X, theta)) - Y
    return (1/(2*X.shape[1])) * np.dot(X.transpose(), diff)
def BGD(theta, X, Y, max_iteration, learn_rate):
    gradient = cal_gradient(theta, X, Y)
    for i in range(max_iteration):
        theta -= learn_rate*gradient
        gradient = cal_gradient(theta, X, Y)
    return theta

In [43]:
# Evaluation with Sigmoid function
learn_accuracy = []
max_iteration = 50
learn_rates = [1e-4, 1e-8]
for learn_rate in learn_rates:
    X0 = np.ones((X_train.shape[1], 1))
    X = np.hstack((X0, X_train.transpose()))
    
    theta = np.hstack((w0, w.transpose())).reshape((X_train.shape[0]+1, 1))
    BGD(theta, X, y_train.reshape((y_train.shape[0], 1)), max_iteration, learn_rate)
    X0 = np.ones((X_val.shape[1], 1))
    X = np.hstack((X0, X_val.transpose()))
    y_val_pred = np.dot(X, theta)
    y_val_pred = sigmoid(y_val_pred)
    for i in range(y_val_pred.shape[0]):
        if y_val_pred[i]<0.5:
            y_val_pred[i]=0
        else:
            y_val_pred[i]=1
    num = 0
    for i in range(y_val_pred.shape[0]):
        if y_val_pred[i] == y_val[i]:
            num += 1
    learn_accuracy.append(num/y_val_pred.shape[0])
best_learn_rate = learn_rates[learn_accuracy.index(max(learn_accuracy))]
print("best learning rate is", best_learn_rate)
print("best accuracy is", max(learn_accuracy))
#test
X0 = np.ones((X_train.shape[1], 1))
X = np.hstack((X0, X_train.transpose()))
theta = np.hstack((w0, w.transpose())).reshape((X_train.shape[0]+1, 1))
BGD(theta, X, y_train.reshape((y_train.shape[0], 1)), max_iteration, best_learn_rate)
X0 = np.ones((X_test.shape[1], 1))
X = np.hstack((X0, X_test.transpose()))
y_test_pred = np.dot(X, theta)
y_test_pred = sigmoid(y_test_pred)
for i in range(y_test_pred.shape[0]):
    if y_test_pred[i]<0.5:
        y_test_pred[i]=0
    else:
        y_test_pred[i]=1
num = 0
for i in range(y_test_pred.shape[0]):
    if y_test_pred[i] == y_test[i]:
        num += 1
print("classification accuracy is", num/y_test_pred.shape[0])

best learning rate is 0.0001
best accuracy is 0.97
classification accuracy is 0.985


(4) **[10 points]** Learning a simple perceptron with **SGD** (using the given initializations $w^{init}$ and $w^{init}_{0}$) based on the training set ($X_{train}$, $y_{train}$): use the training set and the validation set to obtain a good learning rate(you can set the maximum for iterations and try different learning rate); output the learned model and evaluate its performance on the test set with the classification accuracy.

In [130]:
# SGD Implementation
def cal_gradient_SGD(theta, X, Y):
    diff = sigmoid(np.dot(X, theta)) - Y
    return 1/2 * np.dot(X.reshape((X.shape[0], 1)), diff).reshape((X.shape[0], 1))
def SGD(theta, X, Y, max_iteration, learn_rate):
    k = np.random.randint(0, Y.shape[0])
    gradient = cal_gradient_SGD(theta, X[k], Y[k])
    for i in range(max_iteration):
        theta -= learn_rate*gradient
        k = np.random.randint(0, Y.shape[0])
        gradient = cal_gradient_SGD(theta, X[k], Y[k])
    return theta
X0 = np.ones((X_train.shape[1], 1))
X = np.hstack((X0, X_train.transpose()))
max_iteration = 50
learn_rate = 1e-4
theta = np.hstack((w0, w.transpose())).reshape((X_train.shape[0]+1, 1))
k = np.random.randint(0, y_train.shape[0])
gradient = cal_gradient_SGD(theta, X[k], y_train.reshape((y_train.shape[0], 1))[k])
for i in range(max_iteration):
    theta -= learn_rate*gradient
    k = np.random.randint(0, y_train.shape[0])
    gradient = cal_gradient_SGD(theta, X[k], y_train.reshape((y_train.shape[0], 1))[k])

In [143]:
# Evaluation with Sigmoid function
learn_accuracy = []
max_iteration = 50
learn_rates = [1e-4, 1e-8]
for learn_rate in learn_rates:
    X0 = np.ones((X_train.shape[1], 1))
    X = np.hstack((X0, X_train.transpose()))
    theta = np.hstack((w0, w.transpose())).reshape((X_train.shape[0]+1, 1))
    SGD(theta, X, y_train.reshape((y_train.shape[0], 1)), max_iteration, learn_rate)
    X0 = np.ones((X_val.shape[1], 1))
    X = np.hstack((X0, X_val.transpose()))
    y_val_pred = np.dot(X, theta)
    y_val_pred = sigmoid(y_val_pred)
    for i in range(y_val_pred.shape[0]):
        if y_val_pred[i]<0.5:
            y_val_pred[i]=0
        else:
            y_val_pred[i]=1
    num = 0
    for i in range(y_val_pred.shape[0]):
        if y_val_pred[i] == y_val[i]:
            num += 1
    learn_accuracy.append(num/y_val_pred.shape[0])
best_learn_rate = learn_rates[learn_accuracy.index(max(learn_accuracy))]
print("best learning rate is", best_learn_rate)
print("best accuracy is", max(learn_accuracy))
#test
X0 = np.ones((X_train.shape[1], 1))
X = np.hstack((X0, X_train.transpose()))
theta = np.hstack((w0, w.transpose())).reshape((X_train.shape[0]+1, 1))
SGD(theta, X, y_train.reshape((y_train.shape[0], 1)), max_iteration, best_learn_rate)
print(theta)
X0 = np.ones((X_test.shape[1], 1))
X = np.hstack((X0, X_test.transpose()))
y_test_pred = np.dot(X, theta)
y_test_pred = sigmoid(y_test_pred)
for i in range(y_test_pred.shape[0]):
    if y_test_pred[i]<0.5:
        y_test_pred[i]=0
    else:
        y_test_pred[i]=1
num = 0
for i in range(y_test_pred.shape[0]):
    if y_test_pred[i] == y_test[i]:
        num += 1
print("classification accuracy is", num/y_test_pred.shape[0])

best learning rate is 0.0001
best accuracy is 0.945
[[ 4.99396402e-05]
 [-6.55097633e-04]
 [-6.42324333e-04]
 [-7.70599119e-04]
 [-5.42016182e-04]
 [-5.41884620e-04]
 [-4.48406477e-04]
 [-4.08862325e-04]
 [-7.34917993e-04]
 [-6.62351420e-04]
 [-5.89468179e-04]
 [-3.98673031e-04]
 [-3.52484141e-04]
 [-2.20449065e-04]
 [-5.26532954e-04]
 [-4.72711707e-04]
 [-5.00333796e-04]
 [-5.65467094e-04]
 [-7.03165990e-04]
 [-3.58539543e-04]
 [-4.30735116e-04]]
classification accuracy is 0.985


## (b) SVM

(1) **[10 points]** Use the function **‘svm’** in package **‘sklearn’** to do the binary classification. Output the model and evaluate its performance on each dataset with the classification accuracy.

In [567]:
# SVM Implementation
clf = svm.SVC()
clf.fit(X_train.transpose(), y_train)
y_train_pred = clf.predict(X_train.transpose())
y_val_pred = clf.predict(X_val.transpose())
y_test_pred = clf.predict(X_test.transpose())
num_train = 0
num_val = 0
num_test = 0
for i in range(y_train_pred.shape[0]):
    if y_train_pred[i] == y_train[i]:
        num_train += 1
for i in range(y_val_pred.shape[0]):
    if y_val_pred[i] == y_val[i]:
        num_val += 1
for i in range(y_test_pred.shape[0]):
    if y_test_pred[i] == y_test[i]:
        num_test += 1
print("classification accuracy of training set is", num_train/y_train_pred.shape[0])
print("classification accuracy of validation set is", num_val/y_val_pred.shape[0])
print("classification accuracy of test set is", num_test/y_test_pred.shape[0])

classification accuracy of training set is 0.9864285714285714
classification accuracy of validation set is 0.98
classification accuracy of test set is 0.965


## (c) Compare

(1) **[4 points]** Try to compare  models learned from (a)(3), (a)(4) and (b). Write down your explanation and data support.